In [1]:
import math
from tkinter import *
from tkinter import messagebox
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tkinter import messagebox
from pathlib import Path

In [2]:
root = Tk()
root.title("Wuzzuf scraping ")
root.geometry('700x600')
root.resizable(False , False)
root.config(background='whitesmoke')
job = StringVar()
num_pages = StringVar()
df  = "" 

photo = PhotoImage(file = "E:\\Samsung Innovation Campus\\Tasks\Web Scraping\\wuzzuf-squarelogo-1586535330103.png" )
panel = Label(root , image = photo)
panel.place(x= 230 , y =50)


lb1 = Label(text = "Enter a job name   :" , font = ("Arial" , 13 ) ,bg = 'whitesmoke' )
lb1.place(x =130 , y =320)

ent = Entry(width = '25'  ,textvariable = job)
ent.place(x = 300 , y = 327)

lb1 = Label(text = "Enter the number of pages :" , font = ("Arial" , 13 ) ,bg = 'whitesmoke' )
lb1.place(x =80 , y =380)

ent = Entry(width = '25' , textvariable =num_pages )
ent.place(x = 300 , y = 387)



# class wuzzuf:
# #     def init(self):
# #         self.x = 'Welcome to Wuzzuf'

def job2():
    
    job_name = job.get()
    n_of_pages = num_pages.get()
    
    titles=[]
    company=[]
    location=[]
    skill=[]
    country=[]
    new_location=[]
    final_loc=[]
    date=[]
    work_hours=[]
    twon=[]
    city=[]
    final_skill=[]
    experience=[]
    experience_years=[]
    skill2=[]
    links=[]
    job_des=[]
    label=[]
    education_level=[]
    job_req=[]
    vacancies=[]
    num=0
    job_name=job_name.lower()
    name=job_name.split(' ')
    n_of_pages=int(n_of_pages)-1
#     for i in range (3):
#         num=0
    url='https://wuzzuf.net/search/jobs/?a=navbg&q='+str(name[0])+'%20'+str(name[1])+'&start='+str(num)
    result=requests.get(url)
    src=result.content
    soup=BeautifulSoup(src,'lxml')
    page_limit=int(soup.find('strong').text)
    if(int(n_of_pages)>page_limit//15):
        print('Sorry,Limited Pages,maximum pages you can get is ',(page_limit//15)+1)
    else:
        while True:
            

            print('page num',page_limit)
            if(num>int(n_of_pages)):
                print('endeed')
                break
            job_titles=soup.find_all('h2',{'class':'css-m604qf'})
            company_names=soup.find_all('a',{'class':'css-17s97q8'})
            locations=soup.find_all('span',{'class':'css-5wys0k'})
            time=soup.find_all('div',{'class':'css-1lh32fc'})
            skill_descr=soup.find_all('div',{'class':'css-y4udm8'})
            posted_time=soup.find_all('div',{'class':'css-d7j1kk'})
            print(len(job_titles))
            for j in range (len(job_titles)):
                titles.append(job_titles[j].text)
                links.append('https://wuzzuf.net/'+job_titles[j].find('a').attrs['href'])
                company.append(company_names[j].text[:-2])
                location.append(locations[j].text)
                date.append(posted_time[j].text)
                work_hours.append(time[j].text)
                skill.append(skill_descr[j].text)
                label.append(job_name)
            num+=1
            print(num)
            print('Page switched')
        for link in links:
#             print(link)
            try:
                headers = {
                        "Connection": "keep-alive",
                        "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
                    }
                result2=requests.get(link,headers=headers,timeout=5)
                src2=result2.content
                soup2=BeautifulSoup(src2,'lxml')
            #     salaries=soup2.find('span',{'class':'css-4xky9y'})
                description=soup2.find("span", attrs={"itemprop": "description"})
                edu_level=soup2.find("dd", attrs={"class": "requirement-content"})
                job_r=soup2.find("span", attrs={"itemprop": "responsibilities"})
                job_des.append(description.text)
                education_level.append(edu_level.text)
                job_req.append(job_r.text)
                print("Done")
            except:
                job_des.append('No Job Description')
                job_req.append('No Job Requirement')
                
            
        for i in range (len(location)):
                country.append(location[i].split(',')[-1])
                new_location.append(location[i].split(',')[:-1])
        for i in range(len(new_location)):
            final_loc.append(' '.join(new_location[i]))
        
        for i in range(len(skill)):
            skill[i]=skill[i].lstrip(work_hours[i])
            date[i]=date[i].replace(location[i],'')
            date[i]=date[i].replace(company[i],'')        
        for i in range (len(date)):
            date[i]=date[i][3:]
        
        for i in range (len(titles)):
            skill[i].split('·')
            skill[i]=skill[i].replace('Yrs of Exp','')
            experience.append(skill[i].split('·')[0].strip())
            experience_years.append(skill[i].strip().split('·')[1])
            skill2.append(skill[i].split('·')[2:])
        for i in range(len(new_location)):
            final_skill.append(' '.join(skill2[i]))
            if(len(location[i].split(','))==2):
                twon.append('No twon avaliable')
                city.append(location[i].split(',')[0].strip())
            else:
                twon.append(location[i].split(',')[0].strip())
                city.append(location[i].split(',')[1].strip())
                
        global  df        
        df=pd.DataFrame({'job_name':label,'job_title':titles,'company_name':company,'town':twon,'city':city,'country':country,'posted_date':date,'work_hours':work_hours,'career_level':experience,'experience_yreas':experience_years,'job_description':job_des,'job_requirement':job_req,'skills':final_skill})
        messagebox.showinfo("Done" )
        return df

    
def save ():
    filepath = Path("E:\Samsung Innovation Campus\Tasks\Web Scraping\\data1.csv")  
    # filepath.parent.mkdir(parents=True, exist_ok=True) 
    global  df        

    df.to_csv(filepath,index=False)
    print("OK")


bt1 = Button(text = "Scrap" , font = ("Arial" , 15 ) ,bg = 'white' , fg = 'black',width = '11'  ,command = job2 )
bt1.place(x =200 , y = 450)

bt2 = Button(text = "Save" , font = ("Arial" , 15 ) ,bg = 'white' , fg = 'black' ,width = '11'  , command =save)
bt2.place(x =400 , y = 450)

root.mainloop()

page num 41
15
1
Page switched
page num 41
endeed
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
